Below is the training of classifier for each choice of c value using the dual SVM (Gaussian Kernels) with slack formation

optimization problem: min (lambda) 1/2Qlambda^2 - lambda where Q = yyK
                      such that: 
                        - sum(lambda*y) = 0
                        - lambda <= c
                        - (-lambda) <= 0
                    
b = yi - sum(lambdaj * yj * k(xi, xj))
fx = sum(lambdaj * yi * k(x, xi)) + b

File Paths:  
- training data: spam_data/spam_train.data
- validation data: spam_data/spam_validation.data
- test data: spam_data/spam_test.data

In [12]:
import numpy as np
import cvxopt
import cvxopt.solvers

#read file and store data to X and Y 
def read_data(filename): 
    file = open(filename, 'r', encoding='utf-8-sig')
    dataset = []
    for line in file:
        data = line.split(',')
        x_data = [float(x) for x in data[:57]] 
        y_data = int(data[57]) 
        dataset.append((x_data, y_data))

    X = np.array([x for x, y in dataset])
    Y = np.array([y for x, y in dataset])
    Y = np.where(Y == 0, -1, Y) #replace 0 with -1

    return X, Y

#gaussian kernel equation k(x, z)
def g_kernel(x, z, s): 
    return np.exp(-np.linalg.norm(x-z)**2/(2*(s**2)))

#train data to get optimal lagrange multiplier and b
def train(c, s): 
    train_X, train_Y = read_data("spam_data/spam_train.data")
    num_of_data, _ = train_X.shape

    #Gaussian Matrix
    K = np.zeros((num_of_data, num_of_data))
    for i in range(num_of_data): 
        for j in range(num_of_data):
            K[i, j] = g_kernel(train_X[i], train_X[j], s)

    #P_matrix
    P_matrix = np.outer(train_Y, train_Y) * K

    #q_vector
    q_vector = -np.ones(num_of_data)

    #G_matrix
    G_matrix = np.identity(num_of_data)
    G_matrix = np.vstack((G_matrix, -np.identity(num_of_data)))

    #h_vector
    h_vector = np.ones(num_of_data) * c
    h_vector = np.hstack((h_vector, np.zeros(num_of_data)))

    P = cvxopt.matrix(P_matrix)
    q = cvxopt.matrix(q_vector)
    G = cvxopt.matrix(G_matrix)
    h = cvxopt.matrix(h_vector) 
    A = cvxopt.matrix(train_Y.reshape(1, num_of_data) * 1.) 
    b = cvxopt.matrix(0.0) 

    cvxopt.solvers.options['show_progress'] = False #disable progess to console 
    sol = cvxopt.solvers.qp(P, q, G, h, A, b) 
    lagrange_multiplier = np.array(sol['x'])
    
    #get support vectors 
    sv_indices = np.where(lagrange_multiplier > 1e-5)[0]
    sv_lagrange_multiplier = lagrange_multiplier[sv_indices]
    support_vectors = train_X[sv_indices]
    sv_y = train_Y[sv_indices]

    return sv_indices, sv_lagrange_multiplier, support_vectors, sv_y, K


#main(): 
X, Y = read_data("spam_data/spam_validation.data")
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
sigma = [0.001, 0.01, 0.1, 1, 10, 100]    

for c in C: 
    for s in sigma: 
        correct_predictions = 0
        total_data = X.shape[0]
        sv_indices, sv_lagrange_multiplier, support_vectors, sv_y, K = train(c, s)
        sv_y = sv_y.reshape(sv_y.shape[0], 1)
    
        #compute b
        b = 0.0
        for i in range(len(sv_indices)):
            b += sv_y[i] 
            b -= np.sum(sv_lagrange_multiplier * sv_y * K[sv_indices, :][i])
        b /= len(sv_indices)

        #prediction
        for x, y in zip(X, Y):
            sum = 0
            for i in range(len(sv_indices)):
                sum += sv_lagrange_multiplier[i] * sv_y[i] * g_kernel(support_vectors[i], x, s)
            f_x = sum + b
    
            result = y * f_x
            if (result > 0): #correct classification 
                correct_predictions += 1
            
        accuracy = (correct_predictions / total_data) * 100
        print("c:", c)
        print("sigma:", s)
        print("accuracy:", accuracy, "%")

c: 0.001
sigma: 0.001
accuracy: 61.0 %
c: 0.001
sigma: 0.01
accuracy: 61.0 %
c: 0.001
sigma: 0.1
accuracy: 61.0 %
c: 0.001
sigma: 1
accuracy: 61.0 %
c: 0.001
sigma: 10
accuracy: 61.0 %
c: 0.001
sigma: 100
accuracy: 61.0 %
c: 0.01
sigma: 0.001
accuracy: 61.0 %
c: 0.01
sigma: 0.01
accuracy: 61.0 %
c: 0.01
sigma: 0.1
accuracy: 61.0 %
c: 0.01
sigma: 1
accuracy: 61.0 %
c: 0.01
sigma: 10
accuracy: 70.75 %
c: 0.01
sigma: 100
accuracy: 65.375 %
c: 0.1
sigma: 0.001
accuracy: 61.5 %
c: 0.1
sigma: 0.01
accuracy: 62.0 %
c: 0.1
sigma: 0.1
accuracy: 62.25000000000001 %
c: 0.1
sigma: 1
accuracy: 63.125 %
c: 0.1
sigma: 10
accuracy: 76.75 %
c: 0.1
sigma: 100
accuracy: 62.375 %
c: 1
sigma: 0.001
accuracy: 64.75 %
c: 1
sigma: 0.01
accuracy: 66.25 %
c: 1
sigma: 0.1
accuracy: 68.5 %
c: 1
sigma: 1
accuracy: 71.75 %
c: 1
sigma: 10
accuracy: 82.375 %
c: 1
sigma: 100
accuracy: 61.0 %
c: 10
sigma: 0.001
accuracy: 64.75 %
c: 10
sigma: 0.01
accuracy: 66.375 %
c: 10
sigma: 0.1
accuracy: 68.5 %
c: 10
sigma: 1
accur